In [ ]:
import pymysql
import pandas as pd
import passwords
import sys, os

mysqlip = passwords.mysqlip
mysqlpassword = passwords.mysqlpassword
mysqlusername = passwords.mysqlusername
mysqldb = passwords.mysqldb


counties = [
    'lawrence',
    'butler',
    'beaver',
    'allegheny'
]

In [ ]:
def sendSQLCommand(sql):
    try:
        connection = pymysql.connect(host=mysqlip,
                                     user=mysqlusername,
                                     password=mysqlpassword,
                                     db=mysqldb,
                                     charset='utf8mb4',
                                     cursorclass=pymysql.cursors.DictCursor)

        cursor = connection.cursor()
        cursor.execute(sql)
        dockets = cursor.fetchall()
        cursor.close()
        return dockets
    except Exception as e:
        print(e)
        exc_type, exc_obj, exc_tb = sys.exc_info()
        fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
        print(exc_type, fname, exc_tb.tb_lineno)

In [ ]:
#Cases Fully Removed (Unknown whether by expungement or sealing)
for county in counties:
    sql = '''SELECT nf.county as County,COUNT(*) as `Cases` FROM pennsylvania2022.not_found as nf 
    INNER JOIN pennsylvania.''' + county + '''_cases as oc ON nf.docketNumber = oc.docketNumber
    WHERE YEAR(filingDate) >= '2005'
    GROUP BY County'''
    caseseligible = sendSQLCommand(sql)
    df = pd.DataFrame(caseseligible)
    display(df)


In [ ]:
#Total Cases in Each County
final = []
for county in counties:
    sql = "SELECT COUNT(*) FROM pennsylvania2022." + county + "_cases"
    answer = sendSQLCommand(sql)
    print(county,answer)
    final.append({'County':county.capitalize(),'Cases':answer[0]['COUNT(*)']})

df = pd.DataFrame(final)
display(df.sort_values(by=['Cases'],ascending=False))
title = 'Common Pleas Criminal Cases\n in Database By County'
temp = df.sort_values(by=['Cases'],ascending=False).plot(kind="bar",x='County',ylabel="Amount of Cases",title= title)
temp.figure.savefig("General Graphs/" + title.replace("\n","") + ".png",bbox_inches="tight")

In [ ]:
#Total Charges in Each County
final = []
for county in counties:
    sql = "SELECT COUNT(*) FROM pennsylvania2022." + county + "_charges"
    answer = sendSQLCommand(sql)
    print(county,answer)
    final.append({'County':county.capitalize(),'Charges':answer[0]['COUNT(*)']})

df = pd.DataFrame(final)
display(df.sort_values(by=['Charges'],ascending=False))
temp = df.sort_values(by=['Charges'],ascending=False).plot(kind="bar",x='County',ylabel="Number of Charges",title='Criminal Common Pleas Charges in Database By County')
temp.figure.savefig("General Graphs/ChargesByCounty.png",bbox_inches="tight")

In [ ]:
#Total Cases in Each County (With DISTINCT DocketNumber)
final = []
for county in counties:
    sql = "SELECT COUNT(DISTINCT(docketNumber)) as Cases FROM pennsylvania2022." + county + "_charges"
    answer = sendSQLCommand(sql)
    print(county,answer)
    final.append({'County':county.capitalize(),'Cases':answer[0]['Cases']})

df = pd.DataFrame(final)
display(df.sort_values(by=['Cases'],ascending=False))
title = 'Common Pleas Criminal Cases in Database By County'
temp = df.sort_values(by=['Cases'],ascending=False).plot(kind="bar",x='County',ylabel="Number of Cases",xlabel='County',title=title)
#temp.figure.savefig("General Graphs/" + title + ".png",bbox_inches="tight")

In [ ]:
#Total cases grouped by year
for county in counties:
    sql = "SELECT YEAR(filingDate) as 'Filing Date', COUNT(*) as Cases FROM pennsylvania2022." + county + "_cases GROUP BY YEAR(filingDate) ORDER BY YEAR(filingDate) DESC"
    answer = sendSQLCommand(sql)
    print(answer)

    # newanswer = []
    # for temp in answer:
    #     newanswer.append({'Filing Date':str(temp['Filing Date']),'Cases':temp['Cases']})
    df = pd.DataFrame(answer)
    df['Filing Date'] = df['Filing Date'].astype(int)
    df = df.loc[(df["Filing Date"] >= 1990)]
    display(df.sort_values('Filing Date',ascending=True).head(30))
    title = 'Common Pleas Criminal Cases in \n' + county.capitalize() + ' County By Filing Date'
    temp = df.sort_values(by=['Filing Date'],ascending=True).head(30).plot(kind="bar",x='Filing Date',xlabel='Filing Year',ylabel="Amount of Cases",title=title)
    temp.figure.savefig("General Graphs/" + title.replace("\n","") +".png",bbox_inches="tight")
    

In [ ]:
#Total charges grouped by year
for county in counties:
    sql = "SELECT YEAR(filingDate) as 'Filing Date', COUNT(*) as Charges FROM pennsylvania2022." + county + "_cases as dbcases INNER JOIN pennsylvania2022." + county + "_charges as dbcharges on dbcases.docketNumber = dbcharges.docketNumber GROUP BY YEAR(filingDate) ORDER BY YEAR(filingDate) DESC"
    answer = sendSQLCommand(sql)
    print(answer)

    # newanswer = []
    # for temp in answer:
    #     newanswer.append({'Filing Date':str(temp['Filing Date']),'Cases':temp['Cases']})
    df = pd.DataFrame(answer)
    df['Filing Date'] = df['Filing Date'].astype(int)
    df = df.loc[(df["Filing Date"] >= 1990)]
    display(df.sort_values('Filing Date',ascending=True).head(30))
    temp = df.sort_values(by=['Filing Date'],ascending=True).head(30).plot(kind="bar",x='Filing Date',ylabel="Number of Cases",title='Common Pleas Criminal Charges in ' + county.capitalize() + ' County By Filing Date')
    temp.figure.savefig("General Graphs/" + county.capitalize() + " Charges by Filing Date[Old].png",bbox_inches="tight")

In [ ]:
#Dispositions by county in closed cases

for county in counties:
    print("Grabbing",county)
    sql = '''
    SELECT disposition,COUNT(*) as Dispositions FROM pennsylvania2022.''' + county + '''_charges as ch INNER JOIN pennsylvania2022.''' + county + '''_cases as cs ON ch.docketNumber = cs.docketNumber
    WHERE cs.status IN ('Closed','Adjudicated','Adjudicated/Closed') GROUP BY disposition ORDER BY COUNT(*) DESC LIMIT 15
    '''
    
    dispositions = sendSQLCommand(sql)
    df = pd.DataFrame(dispositions)
    display(df)
    title = 'Disposition Types in Closed Common Pleas \nCriminal Cases in ' + county.capitalize() + ' County'
    temp = df.sort_values(by=['Dispositions'],ascending=False).head(15).plot(kind="bar",x='disposition',xlabel = 'Disposition Type',ylabel="Amount of Dispositions",title=title)
    temp.figure.savefig("General Graphs/" + title.replace("\n","") + ".png",bbox_inches="tight")

In [ ]:
#Favorable Dispositions in closed cases
final = []
for county in counties:
    print("Grabbing",county)
    sql = '''
    SELECT disposition,COUNT(*) as Dispositions FROM pennsylvania2022.''' + county + '''_charges as ch INNER JOIN pennsylvania2022.''' + county + '''_cases as cs ON ch.docketNumber = cs.docketNumber
    WHERE cs.status IN ('Closed','Adjudicated','Adjudicated/Closed') GROUP BY disposition ORDER BY COUNT(*) DESC
    '''
    
    dispositions = sendSQLCommand(sql)
    print(dispositions)
    for item in dispositions:
        found = False
        if(item['disposition'] != None and 'guilty' in item['disposition'].lower() and item['disposition'].lower().startswith("not guilty") == False):
            continue
        for x,item2 in enumerate(final):
            if(item['disposition'] == item2['disposition']):
                final[x]['Dispositions'] += item['Dispositions']
                found = True
        if(found == False):
            final.append(item)

print("--------")
print(final)

df = pd.DataFrame(final)
display(df.sort_values('Dispositions',ascending=False))
title = 'Non-Guilty Disposition Types in \nClosed Common Pleas Criminal Cases In All Four Counties'
temp = df.sort_values(by=['Dispositions'],ascending=False).head(15).plot(kind="bar",x='disposition',xlabel = 'Disposition Type',ylabel="Amount of Dispositions",title=title)
temp.figure.savefig("General Graphs/" + title.replace("\n","") + ".png",bbox_inches="tight")

In [ ]:
#Combined Closed Charges with Favorable Dispositions
#Note the majority of these are "Quashed, Dismissed, Demurrer Sustained" from Alleghany which we count as favorable.
final = []
for x in range(1990,2021):
    final.append({'Filing Date':x,'Dispositions':0})
print(final)

for county in counties:
    print("Grabbing",county)
    sql = '''
    SELECT YEAR(filingDate) as 'Filing Date',COUNT(*) as Dispositions FROM pennsylvania2022.''' + county + '''_charges as ch INNER JOIN pennsylvania2022.''' + county + '''_cases as cs ON ch.docketNumber = cs.docketNumber
    WHERE cs.status IN ('Closed','Adjudicated','Adjudicated/Closed') AND disposition IS NOT NULL and YEAR(filingDate) >= 1990 
    AND (ch.disposition like "%nolle%" or ch.disposition LIKE "%dismiss%" or ch.disposition like "%withdraw%" or SUBSTRING(ch.disposition, 1, 10) = "not guilty")
    GROUP BY YEAR(filingDate) ORDER BY COUNT(*)
    '''
    
    dispositions = sendSQLCommand(sql)
    df = pd.DataFrame(dispositions)
    display(df)
    print(dispositions)
    for item in dispositions:
        for x,item2 in enumerate(final):
            if(item['Filing Date'] == item2['Filing Date']):
                final[x]['Dispositions'] += item['Dispositions']
    title = 'Non-Conviction Disposition in Closed Common \nPleas Criminal Cases in ' + county.capitalize() + ' County'
    temp = df.sort_values(by=['Filing Date'],ascending=True).plot(kind="bar",x='Filing Date',xlabel = 'Filing Date',ylabel="Amount of Dispositions",title=title)
    temp.figure.savefig("General Graphs/" + title.replace("\n","") + ".png",bbox_inches="tight")

print("--------")
print(final)

df = pd.DataFrame(final)
display(df.sort_values('Filing Date',ascending=True))
title = 'Non-Conviction Disposition in Closed \nCommon Pleas Criminal Cases In All Four Counties'
temp = df.sort_values(by=['Filing Date'],ascending=True).plot(kind="bar",x='Filing Date',xlabel = 'Filing Year',ylabel="Amount of Dispositions",title=title)
temp.figure.savefig("General Graphs/" + title.replace("\n","") +".png",bbox_inches="tight")


In [ ]:
#None Dispositions Charges by county in closed cases
final = []
for x in range(1990,2021):
    final.append({'Filing Date':x,'Dispositions':0})

for county in counties:
    print("Grabbing",county)
    sql = '''
    SELECT YEAR(filingDate) as 'Filing Date',COUNT(*) as Dispositions FROM pennsylvania2022.''' + county + '''_charges as ch INNER JOIN pennsylvania2022.''' + county + '''_cases as cs ON ch.docketNumber = cs.docketNumber
    WHERE cs.status IN ('Closed','Adjudicated','Adjudicated/Closed') AND disposition IS NULL and YEAR(filingDate) >= 1990 GROUP BY YEAR(filingDate) ORDER BY COUNT(*)
    '''
    
    dispositions = sendSQLCommand(sql)
    df = pd.DataFrame(dispositions)
    display(df)
    title = 'Charges with No Disposition in Closed Common \nPleas Criminal Cases in ' + county.capitalize() + ' County'
    temp = df.sort_values(by=['Filing Date'],ascending=True).plot(kind="bar",x='Filing Date',xlabel = 'Filing Date',ylabel="Number of None Dispositions",title=title)
    #temp.figure.savefig("General Graphs/"+ title + ".png",bbox_inches="tight")
    
    for item in dispositions:
            for x,item2 in enumerate(final):
                if(item['Filing Date'] == item2['Filing Date']):
                    final[x]['Dispositions'] += item['Dispositions']

                    
print("--------")
print(final)

df = pd.DataFrame(final)
display(df.sort_values('Filing Date',ascending=True))
title = 'Charges with No Disposition in Closed \nCommon Pleas Criminal Case In All Four Counties'
temp = df.sort_values(by=['Filing Date'],ascending=True).plot(kind="bar",x='Filing Date',xlabel = 'Filing Year',ylabel="Amount of Dispositions",title=title)
temp.figure.savefig("General Graphs/" + title.replace("\n","") +".png",bbox_inches="tight")
                    




In [ ]:
#Withdrawn Dispositions by county in closed cases
for county in counties:
    print("Grabbing",county)
    sql = '''
    SELECT YEAR(filingDate) as 'Filing Date',COUNT(*) as Dispositions FROM pennsylvania2022.''' + county + '''_charges as ch INNER JOIN pennsylvania2022.''' + county + '''_cases as cs ON ch.docketNumber = cs.docketNumber
    WHERE cs.status IN ('Closed','Adjudicated','Adjudicated/Closed') AND disposition LIKE '%withdraw%' and YEAR(filingDate) >= 1990 GROUP BY YEAR(filingDate) ORDER BY COUNT(*)
    '''
    
    dispositions = sendSQLCommand(sql)
    df = pd.DataFrame(dispositions)
    display(df)
    temp = df.sort_values(by=['Filing Date'],ascending=True).plot(kind="bar",x='Filing Date',xlabel = 'Filing Date',ylabel="Number of Withdrawn Dispositions",title='Withdrawn Dispositions in Closed Common Pleas Criminal Cases in ' + county.capitalize() + ' County')
    #temp.figure.savefig("General Graphs/" + county.capitalize() + " None Dispositions By Filing Year Closed Cases.png",bbox_inches="tight")




In [ ]:
#Withdrawn Dispositions Cases by county in closed cases
for county in counties:
    print("Grabbing",county)
    sql = '''
    SELECT YEAR(filingDate) as 'Filing Date',COUNT(DISTINCT(cs.docketNumber)) as Dispositions FROM pennsylvania2022.''' + county + '''_charges as ch INNER JOIN pennsylvania2022.''' + county + '''_cases as cs ON ch.docketNumber = cs.docketNumber
    WHERE cs.status IN ('Closed','Adjudicated','Adjudicated/Closed') AND disposition LIKE '%withdraw%' and YEAR(filingDate) >= 1990 GROUP BY YEAR(filingDate) ORDER BY COUNT(*)
    '''
    
    dispositions = sendSQLCommand(sql)
    df = pd.DataFrame(dispositions)
    display(df)
    temp = df.sort_values(by=['Filing Date'],ascending=True).plot(kind="bar",x='Filing Date',xlabel = 'Filing Date',ylabel="Number of Withdrawn Dispositions",title='Cases with at least one Withdrawn Disposition in Closed Common Pleas Criminal Cases in ' + county.capitalize() + ' County')
    #temp.figure.savefig("General Graphs/" + county.capitalize() + " None Dispositions By Filing Year Closed Cases.png",bbox_inches="tight")




In [ ]:
#Dispositions of Quashed, Dismissed, Demurrer Sustained

# for county in counties:
#     print("Grabbing",county)
#     sql = '''
#     SELECT YEAR(filingDate) as 'Filing Date',COUNT(*) as Dispositions FROM pennsylvania2022.''' + county + '''_charges as ch INNER JOIN pennsylvania2022.''' + county + '''_cases as cs ON ch.docketNumber = cs.docketNumber
#     WHERE (cs.status = 'Closed' or cs.status = 'Adjudicated') AND disposition = 'Quashed, Dismissed, Demurrer Sustained' and YEAR(filingDate) >= 1990 GROUP BY YEAR(filingDate) ORDER BY COUNT(*)
#     '''
    
#     dispositions = sendSQLCommand(sql)
#     df = pd.DataFrame(dispositions)
#     display(df)
#     title = 'Quashed, Dismissed, Demurrer Sustained Dispositions in Closed Common Pleas Criminal Cases in ' + county.capitalize() + ' County'
#     temp = df.sort_values(by=['Filing Date'],ascending=True).plot(kind="bar",x='Filing Date',xlabel = 'Filing Date',ylabel="Number of Withdrawn Dispositions",title = title)
#     #temp.figure.savefig("General Graphs/" + county.capitalize() + " None Dispositions By Filing Year Closed Cases.png",bbox_inches="tight")


#For All Counties
sql = '''
SELECT SUM(Dispositions) as Dispositions,`Filing Date` FROM (
SELECT YEAR(filingDate) as 'Filing Date',COUNT(*) as Dispositions FROM pennsylvania2022.lawrence_charges as ch INNER JOIN pennsylvania2022.lawrence_cases as cs ON ch.docketNumber = cs.docketNumber
    WHERE cs.status IN ('Closed','Adjudicated','Adjudicated/Closed') AND disposition = 'Quashed, Dismissed, Demurrer Sustained' and YEAR(filingDate) >= 1990 GROUP BY YEAR(filingDate)
 UNION
 SELECT YEAR(filingDate) as 'Filing Date',COUNT(*) as Dispositions FROM pennsylvania2022.beaver_charges as ch INNER JOIN pennsylvania2022.beaver_cases as cs ON ch.docketNumber = cs.docketNumber
    WHERE cs.status IN ('Closed','Adjudicated','Adjudicated/Closed') AND disposition = 'Quashed, Dismissed, Demurrer Sustained' and YEAR(filingDate) >= 1990 GROUP BY YEAR(filingDate)
     UNION
 SELECT YEAR(filingDate) as 'Filing Date',COUNT(*) as Dispositions FROM pennsylvania2022.butler_charges as ch INNER JOIN pennsylvania2022.butler_cases as cs ON ch.docketNumber = cs.docketNumber
    WHERE cs.status IN ('Closed','Adjudicated','Adjudicated/Closed') AND disposition = 'Quashed, Dismissed, Demurrer Sustained' and YEAR(filingDate) >= 1990 GROUP BY YEAR(filingDate)
     UNION
 SELECT YEAR(filingDate) as 'Filing Date',COUNT(*) as Dispositions FROM pennsylvania2022.allegheny_charges as ch INNER JOIN pennsylvania2022.allegheny_cases as cs ON ch.docketNumber = cs.docketNumber
    WHERE cs.status IN ('Closed','Adjudicated','Adjudicated/Closed') AND disposition = 'Quashed, Dismissed, Demurrer Sustained' and YEAR(filingDate) >= 1990 GROUP BY YEAR(filingDate)
 )as temp GROUP BY `Filing Date` ORDER BY `Filing Date` DESC'''

dispositions = sendSQLCommand(sql)
print(dispositions)
#Check if all years are represented and add in 0 years if they are not.
years={1990:False,1991:False,1992:False,1993:False,1994:False,1995:False,1996:False,1997:False,1998:False,1999:False,
       2000:False,2001:False,2002:False,2003:False,2004:False,2005:False,2006:False,2007:False,2008:False,2009:False,
       2010:False,2011:False,2012:False,2013:False,2014:False,2015:False,2016:False,2017:False,2018:False,2019:False,2020:False}
for row in dispositions:
    if(row['Filing Date'] in years.keys()):
        years[row['Filing Date']] = True
for year,value in years.items():
    if(value == False):
        dispositions.append({'Filing Date':year,'Dispositions':0})
df = pd.DataFrame(dispositions)
df["Dispositions"] = pd.to_numeric(df["Dispositions"])
display(df)


title = 'Quashed, Dismissed, Demurrer Sustained Dispositions in \nClosed Common Pleas Criminal Cases in All Four Counties'
temp = df.sort_values(by=['Filing Date'],ascending=True).plot(kind="bar",x='Filing Date',xlabel = 'Filing Year',ylabel="Amount of Dispositions",title = title)
temp.figure.savefig("General Graphs/" + title.replace("\n","") + ".png",bbox_inches="tight")


In [ ]:
#Migrated Dispositions by county in closed cases
for county in counties:
    print("Grabbing",county)
    sql = '''
    SELECT YEAR(filingDate) as 'Filing Date',COUNT(*) as Dispositions FROM pennsylvania2022.''' + county + '''_charges as ch INNER JOIN pennsylvania2022.''' + county + '''_cases as cs ON ch.docketNumber = cs.docketNumber
    WHERE cs.status IN ('Closed','Adjudicated','Adjudicated/Closed') AND disposition LIKE '%migrated%' and YEAR(filingDate) >= 1990 GROUP BY YEAR(filingDate) ORDER BY COUNT(*)
    '''
    #print(sql)
    dispositions = sendSQLCommand(sql)
    print(dispositions)
    if(not dispositions):
        #Some counties don't have any migrated cases
        continue
    #Check if all years are represented and add in 0 years if they are not.
    years={1990:False,
           1991:False,
           1992:False,
           1993:False,
           1994:False,
           1995:False,
           1996:False,
           1997:False,
           1998:False,
           1999:False,
           2000:False,2001:False,2002:False,2003:False,2004:False,2005:False,2006:False,2007:False,2008:False,2009:False,
           2010:False,2011:False,2012:False,2013:False,2014:False,2015:False,2016:False,2017:False,2018:False,2019:False,2020:False}
    for row in dispositions:
        if(row['Filing Date'] in years.keys()):
            years[row['Filing Date']] = True
    for year,value in years.items():
        if(value == False):
            dispositions.append({'Filing Date':year,'Dispositions':0})
    df = pd.DataFrame(dispositions)
    display(df)
    title = 'Migrated Dispositions in Closed Common\n Pleas Criminal Cases in ' + county.capitalize() + ' County'
    temp = df.sort_values(by=['Filing Date'],ascending=True).plot(kind="bar",x='Filing Date',xlabel = 'Filing Year',ylabel="Amount of Dispositions",title=title)
    temp.figure.savefig("General Graphs/" + title.replace("\n","") + ".png",bbox_inches="tight")




In [ ]:
#Confusing Dispositions Charges by county in closed cases
final = []
for x in range(1990,2021):
    final.append({'Filing Date':x,'Dispositions':0})

for county in counties:
    print("Grabbing",county)
    sql = '''
    SELECT YEAR(filingDate) as 'Filing Date',COUNT(*) as Dispositions FROM pennsylvania2022.''' + county + '''_charges as ch INNER JOIN pennsylvania2022.''' + county + '''_cases as cs ON ch.docketNumber = cs.docketNumber
    WHERE cs.status IN ('Closed','Adjudicated','Adjudicated/Closed') AND 
    disposition in ('Disposed at Lower Court','Charge Changed','Proceed to Court','Information Filed','Waived for Court (Lower Court)','Held for Court (Lower Court)','Held for Court') 
    and YEAR(filingDate) >= 1990 GROUP BY YEAR(filingDate) ORDER BY COUNT(*)
    '''
    
    dispositions = sendSQLCommand(sql)
    df = pd.DataFrame(dispositions)
    display(df)
    title = 'Charges with a Confusing Disposition in Closed Common Pleas Criminal Cases in ' + county.capitalize() + ' County'
    temp = df.sort_values(by=['Filing Date'],ascending=True).plot(kind="bar",x='Filing Date',xlabel = 'Filing Date',ylabel="Number of Dispositions",title=title)
    #temp.figure.savefig("General Graphs/"+ title + ".png",bbox_inches="tight")
    
    for item in dispositions:
            for x,item2 in enumerate(final):
                if(item['Filing Date'] == item2['Filing Date']):
                    final[x]['Dispositions'] += item['Dispositions']

                    
print("--------")
print(final)

df = pd.DataFrame(final)
display(df.sort_values('Filing Date',ascending=True))
title = 'Charges with a Confusing Disposition \nin Closed Common Pleas Criminal Cases In All Four Counties'
temp = df.sort_values(by=['Filing Date'],ascending=True).plot(kind="bar",x='Filing Date',xlabel = 'Filing Year',ylabel="Amount of Dispositions",title=title)
temp.figure.savefig("General Graphs/" + title.replace("\n","") +".png",bbox_inches="tight")
                    




In [ ]:
#Favorable Disposition Breakdown
sql = '''
SELECT disposition as Disposition,SUM(Total) as Dispositions FROM (
SELECT disposition,count(*) as Total FROM pennsylvania2022.allegheny_charges as charges INNER JOIN pennsylvania2022.allegheny_cases as cases on charges.docketNumber = cases.docketNumber 
WHERE status in ('Closed','Adjudicated','Adjudicated/Closed') and (disposition LIKE '%nolle%' or disposition like "%dismiss%" or disposition like '%withdraw%' or disposition like 'not guilty%') group by disposition
UNION
SELECT disposition,count(*) as Total FROM pennsylvania2022.butler_charges as charges INNER JOIN pennsylvania2022.butler_cases as cases on charges.docketNumber = cases.docketNumber 
WHERE status in ('Closed','Adjudicated','Adjudicated/Closed') and (disposition LIKE '%nolle%' or disposition like "%dismiss%" or disposition like '%withdraw%' or disposition like 'not guilty%') group by disposition
UNION
SELECT disposition,count(*) as Total FROM pennsylvania2022.beaver_charges as charges INNER JOIN pennsylvania2022.beaver_cases as cases on charges.docketNumber = cases.docketNumber 
WHERE status in ('Closed','Adjudicated','Adjudicated/Closed') and (disposition LIKE '%nolle%' or disposition like "%dismiss%" or disposition like '%withdraw%' or disposition like 'not guilty%') group by disposition
UNION
SELECT disposition,count(*) as Total FROM pennsylvania2022.lawrence_charges as charges INNER JOIN pennsylvania2022.lawrence_cases as cases on charges.docketNumber = cases.docketNumber 
WHERE status in ('Closed','Adjudicated','Adjudicated/Closed') and (disposition LIKE '%nolle%' or disposition like "%dismiss%" or disposition like '%withdraw%' or disposition like 'not guilty%') group by disposition
) as temp GROUP BY Disposition ORDER BY Dispositions DESC
'''
dispositions = sendSQLCommand(sql)
df = pd.DataFrame(dispositions)

print(df.dtypes)
#df['Dispositions'].astype(float)
df["Dispositions"] = pd.to_numeric(df["Dispositions"])
print(df.dtypes)
display(df)
title = 'Non-Conviction Disposition Types in Closed \nCommon Pleas Criminal Case In All Counties'
temp = df.sort_values(by=['Dispositions'],ascending=False).head(10).plot(kind="bar",x='Disposition',xlabel = 'Disposition Type',ylabel="Amount of Dispositions",title=title)
temp.figure.savefig("General Graphs/" + title.replace("\n","") +".png",bbox_inches="tight")